In [1]:
import numpy as np
import pandas as pd 
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
from PIL import Image
from sklearn.utils import resample
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from transformers import RobertaTokenizer
from transformers import TFRobertaModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import Flatten
from sklearn.metrics import precision_recall_curve

import tensorflow as tf
from sklearn.metrics import roc_auc_score, roc_curve,auc
from keras import regularizers
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,f1_score,classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.metrics import average_precision_score
from itertools import cycle

lemma = WordNetLemmatizer()
stopword = set(STOPWORDS)
nltk.download('omw-1.4')
%matplotlib inline
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')


from nltk.corpus import brown
nltk.download("brown")
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


# Import Train Data

# Data preprocessing

In [5]:
%cd PycharmProjects/proj_bach

C:\Users\alexh\PycharmProjects\proj_bach


In [6]:
from data_retriever import data1, data2

File simple_emotions.txt already exists. Not overwriting.
File simple_sentiments.txt already exists. Not overwriting.
File clean_tweet_emotions.txt already exists. Not overwriting.
File clean_tweet_sentiments.txt already exists. Not overwriting.
                                                Text Sentiment
0  During the period of falling in love, each tim...  Positive
1         When I was involved in a traffic accident.  Negative
2  When I was driving home after  several days of...  Negative
3   When I lost the person who meant the most to me.  Negative
4  The time I knocked a deer down - the sight of ...  Negative
Index(['Text', 'Sentiment'], dtype='object')
                                                   Text Sentiment
0     During the period of falling in love, each tim...  Positive
1            When I was involved in a traffic accident.  Negative
2     When I was driving home after  several days of...  Negative
3      When I lost the person who meant the most to me.  Negative
4

In [7]:
def clean_text(tweet):
    tweets = " ".join(filter(lambda x: x[0]!= '@' , tweet.split()))
    tweets = re.sub('[^a-zA-Z]', ' ', tweets)
    tweets = tweets.lower()
    tweets = tweets.split()
    tweets = [word for word in tweets if not word in set(stopwords.words('english'))]
    tweets = [lemma.lemmatize(word) for word in tweets]
    tweets = " ".join(tweets)
    return tweets

In [8]:
train_data = data2.astype(str).copy()

In [9]:
train_data['clean_tweet_text'] = train_data.Text.apply(clean_text)

In [10]:
print(train_data.head())

                                                Text Sentiment  \
0                                                nan   neutral   
1                     Sage Act upgrade list tommorow  Positive   
2  WAY HOMEGIRL BABY FUNERAL MAN HATE FUNERALS SH...  Negative   
3  eye  true hazel eyeand brilliant  Regular feat...  Positive   
4    ugh babe hugggzzz u  babe naamazed nga ako e...  Positive   

                                    clean_tweet_text  
0                                                nan  
1                     sage act upgrade list tommorow  
2  way homegirl baby funeral man hate funeral sho...  
3  eye true hazel eyeand brilliant regular featur...  
4  ugh babe hugggzzz u babe naamazed nga ako e ba...  


In [11]:
train_data=train_data.drop(["Text"],axis=1)

In [12]:
train_data.head()

,Sentiment,clean_tweet_text
0,neutral,nan
1,Positive,sage act upgrade list tommorow
2,Negative,way homegirl baby funeral man hate funeral sho...
3,Positive,eye true hazel eyeand brilliant regular featur...
4,Positive,ugh babe hugggzzz u babe naamazed nga ako e ba...


In [13]:
max_len=128
text_data=train_data["clean_tweet_text"]
label_data=train_data["Sentiment"]

# roberta_tokenizer

In [14]:
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# roberta_model

In [15]:
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(117.27338, shape=(), dtype=float32)


In [16]:
roberta_model = TFRobertaModel.from_pretrained("roberta-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

# Sample Text

In [17]:
text_data[1300]

'time elated vice said mr tyson'

In [18]:
encode_data=roberta_tokenizer.encode_plus(text_data[1300],add_special_tokens = True,max_length =40,pad_to_max_length = True,truncation=True)

In [19]:
encode_data

{'input_ids': [0, 958, 1615, 1070, 2626, 26, 475, 338, 326, 20216, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [20]:
test_input_id=np.asarray(encode_data['input_ids'])
test_attention_mask=np.asarray(encode_data['attention_mask'])


In [21]:
print(test_input_id.shape)
print(test_attention_mask.shape)

print(test_attention_mask.reshape(1,-1).shape)

(40,)
(40,)
(1, 40)


In [22]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [23]:
import transformers

In [24]:
print(isinstance(roberta_model, transformers.TFRobertaModel))  # Should return True


True


In [25]:
output_data=roberta_model([test_input_id.reshape(1,-1),test_attention_mask.reshape(1,-1)])

type(output_data)

transformers.modeling_tf_outputs.TFBaseModelOutputWithPoolingAndCrossAttentions

In [26]:
output_data

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 40, 768), dtype=float32, numpy=
array([[[-0.06151576,  0.06928044, -0.02511747, ..., -0.11403868,
         -0.06508613, -0.01430185],
        [-0.22992018,  0.00537658, -0.2360584 , ..., -0.31573626,
         -0.1950437 ,  0.00210309],
        [ 0.07679282,  0.1645904 ,  0.03715213, ...,  0.05029377,
          0.02517287,  0.10758667],
        ...,
        [ 0.04703602, -0.03357879, -0.10649902, ..., -0.4143144 ,
         -0.0469461 ,  0.00187367],
        [ 0.04703602, -0.03357879, -0.10649902, ..., -0.4143144 ,
         -0.0469461 ,  0.00187367],
        [ 0.04703602, -0.03357879, -0.10649902, ..., -0.4143144 ,
         -0.0469461 ,  0.00187367]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-2.29509100e-01,  3.58325452e-01,  2.69598067e-01,
         2.51303017e-01, -7.85267651e-02, -4.12345260e-01,
         4.80483770e-02,  2.75966763e-01, -2.29786590e-

In [27]:
roberta_tokenizer.decode(encode_data["input_ids"])

'<s>time elated vice said mr tyson</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [28]:
final_text_data=train_data["clean_tweet_text"]
final_label_data=np.array(label_data)

In [29]:
new_final_label_data = np.array([0 if label=="Negative" else 1 if label=="neutral" else 2 for label in label_data])

In [30]:
new_final_label_data = np.array([0 if label=="Negative" else 1 for label in label_data])

In [31]:
print(new_final_label_data[:25])

[1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1]


In [32]:
def roberta_encode(final_text_data,max_len):
    input_ids=[]
    attention_masks=[]
    
    for i in range(len(final_text_data)):
        encode_data=roberta_tokenizer.encode_plus(final_text_data[i],add_special_tokens=True,max_length=max_len,pad_to_max_length=True,return_attention_mask=True)
        input_ids.append(encode_data['input_ids'])
        attention_masks.append(encode_data["attention_mask"])
    return np.array(input_ids),np.array(attention_masks)
        

In [33]:
text_input_ids,text_attention_masks = roberta_encode(final_text_data,max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [34]:
print('Text Input Ids Shape {} \nText Input Attention Mask Shape {} \nLabel Data shape {}'.format(text_input_ids.shape,text_attention_masks.shape,new_final_label_data.shape))

Text Input Ids Shape (30730, 128) 
Text Input Attention Mask Shape (30730, 128) 
Label Data shape (30730,)


In [35]:
X_train_input,X_test_input,Y_train_label,Y_test_label,train_mask,test_mask=train_test_split(text_input_ids,new_final_label_data,text_attention_masks,test_size=0.2,random_state=42,shuffle=True)

In [36]:
print('Train input shape {}\nTest input shape {}\nTrain label shape {}\nTest label shape {}\nTrain attention mask shape {}\nTest attention mask shape {}'.format(X_train_input.shape,X_test_input.shape,Y_train_label.shape,Y_test_label.shape,train_mask.shape,test_mask.shape))

Train input shape (24584, 128)
Test input shape (6146, 128)
Train label shape (24584,)
Test label shape (6146,)
Train attention mask shape (24584, 128)
Test attention mask shape (6146, 128)


In [37]:
print(Y_train_label)

[0 0 0 ... 1 0 1]


# Create Roberta Model

In [ ]:
from tensorflow.keras.layers import LeakyReLU


In [ ]:
def Create_Roberta_Model():
    input_ids=tf.keras.Input(shape=(max_len,),dtype="int32")
    attention_masks=tf.keras.Input(shape=(max_len,),dtype="int32")
    
    roberta_model = TFRobertaModel.from_pretrained("roberta-base")
    output_dim = roberta_model(input_ids=input_ids, attention_mask=attention_masks)[0][:,0,:]
    
    dense_layer = tf.keras.layers.Dense(128, activation="relu")(output_dim)
    dropout = tf.keras.layers.Dropout(0.5)(dense_layer)
    final_layer = tf.keras.layers.Dense(2, activation='softmax')(dropout)

    model = tf.keras.models.Model(inputs=[input_ids, attention_masks], outputs=final_layer)
    
    return model


In [ ]:
model=Create_Roberta_Model()

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

optimizer = tf.keras.optimizers.Adam(lr=1e-5)

model.compile(loss=loss,optimizer=optimizer, metrics=["accuracy"])

In [ ]:
with tf.GradientTape() as tape:
    predictions = model([X_train_input[:1], train_mask[:1]], training=True)
    loss = tf.keras.losses.sparse_categorical_crossentropy(Y_train_label[:1], predictions)
grads = tape.gradient(loss, model.trainable_weights)
for grad, weight in zip(grads, model.trainable_weights):
    if grad is not None:
        print(f"Gradient for {weight.name} is okay")
    else:
        print(f"No gradient for {weight.name}")

In [ ]:
history=model.fit([X_train_input,train_mask],Y_train_label,batch_size=16,epochs=10,validation_data=([X_test_input,test_mask],Y_test_label))

In [ ]:
def model_loss_and_accuracy(history):
    
    fig=plt.figure(figsize=(15,15))
    plt.subplot(221)
    plt.plot(history.history["accuracy"],marker="o",linestyle=":",markersize=10,color="m",label="accuracy")
    plt.plot(history.history["val_accuracy"],marker="D",linestyle=":",markersize=10,color="b",label="val_accuracy")
    plt.title("Model Accuracy\n",fontsize=20,color="darkorange")
    plt.xlabel("Number of Epochs",color="midnightblue",fontsize=15)
    plt.ylabel("Accuracy",color="midnightblue",fontsize=15)
    plt.grid(color = 'green', linestyle = '--', linewidth = 1)
    plt.legend(loc="best")
    plt.tight_layout()
    
  
    plt.subplot(222)
    plt.plot(history.history["accuracy"],marker="v",linestyle="-.",markersize=10,color="g",label="accuracy")
    plt.plot(history.history["val_accuracy"],marker="s",linestyle="--",markersize=10,color="r",label="val_accuracy")
    plt.title("Model Loss\n",fontsize=20,color="limegreen")
    plt.xlabel("Number of Epochs",color="midnightblue",fontsize=15)
    plt.ylabel("Loss",color="midnightblue",fontsize=15)
    plt.grid(color = 'green', linestyle = '--', linewidth = 1)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
model_loss_and_accuracy(history)

# F1 Score

In [ ]:
def plot_f1_score(history):
    f1_scores = []
    for epoch in range(len(history.history['val_loss'])):
        model.set_weights(history.model.get_weights())
        predictions = model.predict([X_test_input, test_mask])
        predictions = [round(p[0]) for p in predictions]
    
        f1_scores.append(f1_score(Y_test_label, predictions, average='weighted'))
    plt.figure(figsize=(10,10))
    plt.plot(range(len(history.history['val_loss'])), f1_scores, label='F1 score',marker="*",color="g",linestyle="--",linewidth=4,markersize=8,markerfacecolor="r")
    plt.title("F1 Score\n",color="black",fontsize=20)
    plt.xlabel('Epochs',fontsize=15,color="black")
    plt.ylabel('F1 score',fontsize=15,color="black")
    plt.legend()
    plt.show()


plot_f1_score(history)


# Label Names

In [ ]:
label_name=["Positive","Racist and Sexist"]

In [ ]:
pred=model.predict([X_test_input, test_mask])
prediction=np.argmax(pred, axis=1)

# Accuracy Score

In [ ]:
print("Accuracy Score is",accuracy_score(Y_test_label,prediction))

# ROC AUC Score

In [ ]:
print("ROC AUC Score is {}".format(roc_auc_score(Y_test_label, pred[:,1])))

# Average Precision Score

In [ ]:
# Get the predicted probabilities for the positive class
pred_probs = model.predict([X_test_input, test_mask])[:,1]

# Binarize the test labels
Y_test_binarized = (Y_test_label == 1).astype(int)

# Calculate the average precision score
average_precision = average_precision_score(Y_test_binarized, pred_probs)

print("Average Precision Score :", average_precision)


# receiver operating characteristic curve

In [ ]:
pred_positive = pred[:,1]
fpr, tpr, thresholds = roc_curve(Y_test_label, pred_positive)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(10,10))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc,color="g",linestyle="--",marker="o",markersize=3,markerfacecolor="k")

plt.plot([0, 1], [0, 1],linestyle="--",linewidth=3,color="m")  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()



In [ ]:
# Calculate precision, recall and thresholds
precision, recall, thresholds = precision_recall_curve(Y_test_label,pred[:,1])

# Plot the PR curve
plt.figure(figsize=(10,10))
plt.step(recall, precision, color='b', alpha=0.2, where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2, color="darkmagenta")
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('Precision-Recall Curve\n',fontsize=20,color="black")
plt.xlabel("Recall",fontsize=15,color="violet")
plt.ylabel("Precision",fontsize=15,color="darkgreen")
plt.show()


In [ ]:
# Encode the labels
label_encoder=LabelEncoder()
label_encoder.fit(Y_test_label)
test_label_data=label_encoder.transform(Y_test_label)
classes=label_encoder.classes_
colors = ["navy", "darkorange"]
plt.figure(figsize=(10,10))

# Compute the ROC curve and AUC for each class
fpr = {}
tpr = {}
roc_auc = dict()
for i, class_ in enumerate(classes):
    # Binarize the label data
    label_binarized = (test_label_data == i).astype(int)
    fpr[i], tpr[i], _ = roc_curve(label_binarized, pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    # Plot the ROC curve for each class
    plt.plot(fpr[i], tpr[i], label='%s ROC Curve Class  (AUC=%0.2f)' % (class_, roc_auc[i]), marker=">", color=colors[i])

plt.plot([0,1],[0,1], marker="H", color="k", linestyle="--", linewidth=4)
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Compute The ROC Curve And AUC For Each Class', color="black", fontsize=15)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()


# Compute The Precision Curve And Area Under Curve For Each Class

In [ ]:
# Encode the labels
label_encoder=LabelEncoder()
label_encoder.fit(Y_test_label)
test_label_data=label_encoder.transform(Y_test_label)
classes=label_encoder.classes_
colors = ["darkred", "purple"]
plt.figure(figsize=(10,10))

# Compute the PR curve and AUC for each class
precision = {}
recall = {}
pr_auc = dict()
for i, class_ in enumerate(classes):
    # Binarize the label data
    label_binarized = (test_label_data == i).astype(int)
    precision[i], recall[i], _ = precision_recall_curve(label_binarized, pred[:, i])
    pr_auc[i] = auc(recall[i], precision[i])

    # Plot the PR curve for each class
    plt.plot(recall[i], precision[i], label='%s Precision Curve Class  (AUC=%0.2f)' % (class_, pr_auc[i]),color=colors[i],marker="D",markersize=2,linestyle="-.")

plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('Compute The Precision Curve And Area Under Curve For Each Class\n',color="darkblue",fontsize=20)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc='lower right')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))

cf_matrix=confusion_matrix(Y_test_label,prediction)
label_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
label_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
label_names = ["True Positive","False Positive","False Negative","True Negative"]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(label_names,label_counts,label_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt="", cmap='gist_ncar',xticklabels=label_name,yticklabels=label_name)
plt.title("Confusion Matrix\n",color="gold",fontsize=20)
plt.ylabel("True Label\n",fontsize=15,color="yellow")
plt.xlabel("\nPredicted Label",fontsize=15,color="yellow")
plt.show()

In [ ]:
print(classification_report(Y_test_label,prediction,target_names=label_name))

# Custom Data Prediction
> * 0==Positive
> * 1==Racist And Sexist

In [ ]:
x="hank you very much for sharing your experience with us.We are really happy that your interaction with our brand was so positive.I just want to let you know that we are acting upon your feedback to make some vital changes to the way we operate [list of changes].As you can see, the opinions of our clients help us to provide better experiences and grow as a company.Regards,"
pred_input=roberta_tokenizer.encode_plus(x,add_special_tokens = True,max_length =128,pad_to_max_length = True,truncation=True)
test_input_id=np.array(pred_input['input_ids'])
test_input_mask=np.array(pred_input['attention_mask'])
# Get the predicted probabilities
text_predict = model.predict([test_input_id.reshape(1,-1), test_input_mask.reshape(1,-1)])

# Get the class with the highest probability
predicted_class = np.argmax(text_predict, axis=-1)[0]

# Check if the input is toxic (1) or positive (0)
if predicted_class == 1:
    print("The input text is toxic.")
else:
    print("The input text is positive.")
